[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xiptos/is_notes/blob/main/rl_cartpole.ipynb)

# Reinforcement Learning

Reinforcement learning problems involve learning what to do, namelly how to map situations to actions, so as to maximize a numerical reward signal.

In this example, let's take a look at the CartPole example.

## Install packages

In [ ]:
!pip install "gymnasium[classic-control]"
!pip install moviepy

## Import libraries

In [ ]:
import gymnasium as gym
import numpy as np
from itertools import count
from collections import deque
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from gymnasium.utils.save_video import save_video
from torch.distributions import Categorical

## Control options

In [ ]:
gamma=0.99
# render="human"
render="rgb_array_list"
log_interval=10
seed=543

env = gym.make('CartPole-v1', render_mode=render)
env.reset(seed=seed)
torch.manual_seed(seed)

## $\pi(s, a)$ neural network

In [ ]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)

## Settings

In [ ]:
policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()

## Functions

In [ ]:
def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(state)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()


# finish_episode() is called at the end of every episode to:
#	1.	Compute the returns (discounted sum of rewards)
#	2.	Normalize them (variance reduction)
#	3.	Compute the policy gradient loss
#	4.	Perform a backward pass and optimize the policy network
#	5.	Clear stored rewards and log-probs for the next episode
def finish_episode():
    R = 0
    policy_loss = []
    returns = deque()
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        returns.appendleft(R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    for log_prob, R in zip(policy.saved_log_probs, returns):
        policy_loss.append(-log_prob * R)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]

def save_episode(episode: int):
    return episode % 10 == 0


In [ ]:
env.metadata["render_fps"]

## Main cicle

In [ ]:
running_reward = 10
step_starting_index = 0
for i_episode in count(1):
    state, _ = env.reset()
    ep_reward = 0
    for t in range(1, 10000):  # Don't infinite loop while learning
        action = select_action(state)
        state, reward, done, _, _ = env.step(action)
        if render=="human":
            env.render()
        policy.rewards.append(reward)
        ep_reward += reward
        if done:
            break

    if render=="rgb_array_list":
        save_video(env.render(),
               "videos", fps=int(env.metadata["render_fps"]), step_starting_index=step_starting_index,
               episode_index=i_episode, episode_trigger=save_episode)
    running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    finish_episode()
    if i_episode % log_interval == 0:
        print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
            i_episode, ep_reward, running_reward))
    if running_reward > env.spec.reward_threshold:
        print("Solved! Running reward is now {} and "
              "the last episode runs to {} time steps!".format(running_reward, t))
        torch.save(policy.state_dict(), 'cartpole_policy.pth')
        break

# Autonomous play...

In [ ]:
import gymnasium as gym
from gymnasium.utils import play
import numpy as np
import sys
import torch
import torch.nn as nn

# Allows for loading and running a pre-trained model,
# or for the user to interact with the environment.
# See 'cartpole_train' for more details.

USER_PLAY = False
MODEL_DIR = "cartpole_policy.pth"

env = gym.make("CartPole-v1", render_mode="human")

if USER_PLAY:
    env.reset()
    key_action_map = {(ord('a'),): 0, (ord('d'),): 1}  # Map 'a' and 'd' to going left and right
    play.play(env, keys_to_action=key_action_map, fps=15)  # Lower fps cause its too hard otherwise
    sys.exit()

# 0: Left, 1: Right
action_size = env.action_space.n
# 0: Cart position, 1: Cart Velocity, 2: Pole Angle, 3: Pole Angular Velocity
state_size = env.observation_space.shape[0]

model = Policy()
model.load_state_dict(torch.load(MODEL_DIR))
model.eval()
n_episodes = 40

for e in range(n_episodes):
    state = env.reset()
    state = np.asarray(state[0])

    done = False
    time_step = 0

    score = 0
    while not done and time_step < 500:
        env.render()

        action = select_action(state)
        next_state, reward, done, info, _ = env.step(action)

        score += reward
        reward = 0.1 if not done else -1
        next_state = np.asarray(next_state)

        state = next_state
        time_step += 1

    print("episode: {}, time steps: {}, score: {}".format(e, time_step, score))